In [15]:
import pymysql
import pymysql.cursors
import cryptography
import json
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import psycopg2
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import configparser
 
config = configparser.ConfigParser()
config.read(r'C:\Users\37529\Desktop\config.ini')

host = config['DATABASE']['host']
port = config['DATABASE']['port']
user = config['DATABASE']['username']
password = config['DATABASE']['password']
database = config['DATABASE']['database']

# Establishing a PYMYSQL connection
def get_conn():
    try:
        connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database,
        charset='utf8mb4',
        cursorclass=pymysql.cursors.Cursor 
    )
        print("Connected successfully")
        return connection
    except pymysql.Error as e:
        logger.error(f'Error connecting to database: {e}')


# Python function to connect to the mysql database 
# and return the sqlalchemy object
def get_connection():
	return create_engine(
		url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
			user, password, host, port, database
		)
	)
try:
    engine = get_connection()
    logger.info(f"Connection to the {host} for user {user} created successfully.")
except pymysql.Error as e:
    logger.error(f"Сonnection couldn't be made due to the following error: {e}")

INFO:__main__:Connection to the localhost for user user created successfully.


In [26]:
"""
Элементарная функция для выполнения запросов,
не требующих возвращения результирующего сета.
"""
def execute_query(query):
    try:
        conn = get_conn()
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()
        logger.info("Запрос выполнен успешно")

    except pymysql.Error as e:
        logger.error(f"Запрос не был выполнен в связи со следующей ошибкой: {e}")

In [27]:
"""
Приведение необходимых sql-запросов для создания бд и двух таблиц в ней.
Создание отношений с помощью добавления первичного и вторичного ключей.
Предполагается исполнение данных запросов посредством описанной выше функции 'execute_query(query)'
"""

create_db_query = """
CREATE DATABASE students_rooms;
"""
create_rooms_table_query = """
CREATE TABLE students_rooms.rooms (
id INT,
name VARCHAR(255),
PRIMARY KEY(id)
);
"""

create_students_table_query = """
CREATE TABLE students_rooms.students (
birthday DATETIME,
id INT,
name VARCHAR(255),
room INT,
sex CHAR(1),
PRIMARY KEY(id),
FOREIGN KEY (room) REFERENCES rooms(id)
);
"""
execute_query(create_db_query)
execute_query(create_rooms_table_query)
execute_query(create_students_table_query)

ERROR:__main__:Запрос не был выполнен в связи со следующей ошибкой: (1007, "Can't create database 'students_rooms'; database exists")
ERROR:__main__:Запрос не был выполнен в связи со следующей ошибкой: (1050, "Table 'rooms' already exists")
ERROR:__main__:Запрос не был выполнен в связи со следующей ошибкой: (1050, "Table 'students' already exists")


Connected successfully
Connected successfully
Connected successfully


In [29]:
def loading_data():
    try:
        engine = get_connection()
        import json
        import pandas as pd
        files_amount = input("How many files do you want to load?")
        for a in range(int(files_amount)):
            file_path = input("Please, provide a path to the file to load")
            with open(file_path.replace('"', ''), 'r') as file:
                file_data = json.load(file)
            df = pd.DataFrame(file_data)
            display(df)
            df.to_sql(input("Enter the name of the table being created"), engine, if_exists='replace', index=False)
            logger.info("The data from the file is successfully loaded to the table")

    except pymysql.Error as e:
        logger.error(f"Data wasn't loaded due to the following error: {e}")  

In [52]:
# ПЕРВЫЙ ЗАПРОС

SELECT
r.id as room_id,
r.name as room_name,
COUNT(s.name) as people_amount_in_a_room
FROM rooms AS r 
LEFT JOIN students AS s 
ON s.room = r.id
GROUP BY r.id, r.name
ORDER BY r.id
;


# ВТОРОЙ ЗАПРОC

SELECT
r.id as room_id,
r.name as room_name,
COUNT(s.name) as amount_of_students_living_in_a_room,
AVG(EXTRACT(YEAR FROM DATE(NOW())) - EXTRACT(YEAR FROM DATE(birthday))) as average_age_in_a_room
FROM rooms as r
LEFT JOIN students as s
ON r.id = s.room 
GROUP BY r.id, r.name
ORDER BY average_age_in_a_room ASC
LIMIT 5;


# ТРЕТИЙ ЗАПРОС

SELECT
r.id,
r.name,
AVG(EXTRACT(YEAR FROM DATE(NOW())) - EXTRACT(YEAR FROM DATE(birthday))) as average_age_in_a_room ,
MAX(EXTRACT(YEAR FROM DATE(NOW())) - EXTRACT(YEAR FROM DATE(birthday))) as the_oldest_in_a_room,
MIN(EXTRACT(YEAR FROM DATE(NOW())) - EXTRACT(YEAR FROM DATE(birthday))) as the_youngest_in_a_room,
MAX(EXTRACT(YEAR FROM DATE(NOW())) - EXTRACT(YEAR FROM DATE(birthday))) - MIN(EXTRACT(YEAR FROM DATE(NOW())) - EXTRACT(YEAR FROM DATE(birthday))) as difference_between_ages
FROM rooms as r
INNER JOIN students as s
ON r.id = s.room
GROUP BY r.id, r.name
ORDER BY difference_between_ages DESC
LIMIT 5;

# ЧЕТВЁРТЫЙ ЗАПРОС

SELECT DISTINCT r.name
FROM rooms r 
INNER JOIN students s ON r.id = s.room 
GROUP BY r.name
HAVING COUNT(DISTINCT s.sex) > 1;

In [31]:
def get_df_resulting_set():
    try:
        import pandas as pd
        engine = get_connection()
        connection = get_conn()
        query = input("Введите запрос:")
        df = pd.read_sql_query(query, engine)
        return df 
        logger.info("Query was executed successfully")

    except pymysql.Error as e:
        logger.error(f"Query wasn't executed due to the following error: {e}") 


def result_to_df(df):
    try:
        display(df)
        return df
        logger.info("Dataframe was returned successfully")

    except pymysql.Error as e:
        logger.error(f"Dataframe wasn't returned due to the following error: {e}") 


def df_save_to_json_or_xml(ready_df):
    try:
        format = input("В каком формате сохранить результат? JSON или XML?")
        if format == "JSON":
            ready_df_json = ready_df.to_json(orient = 'records', indent = 4)
            future_file_name = input("Enter the name of the file we're creating:")
            p = future_file_name.replace('"','')+".json"

            with open(p, "w") as ddd:
                ddd.write(ready_df_json)
            logger.info(f"The file {p} is now present on your PC. Look for it using the name you gave to it") 

        elif format == "XML":
            import lxml
            ready_df_xml = ready_df.to_xml()
            future_file_name = input("Enter the name of the file we're creating:")
            p = future_file_name.replace('"','')+".xml"

            with open(p, "w") as ddd:
                ddd.write(ready_df_xml)
            logger.info(f"The file {p} is now present on your PC. Look for it using the name you gave to it") 
        
        else:
            print("Something weny wrong.")
            
    except pymysql.Error as e:
        logger.error(f"Data wasn't saved to the file due to the following error: {e}")

def full_query_cycle():
    try:
        queries_amount = input("Сколько запросов вы хотите сделать? Введите число")
        for a in range(int(queries_amount)):
            import lxml
            import pandas as pd
            df = get_df_resulting_set()
            ready_df = result_to_df(df)
            df_save_to_json_or_xml(ready_df)
            logger.info("All queries were executed successfully.")
    except pymysql.Error as e:
        logger.error(f"Queries weren't executed successfully due to the following error: {e}")